NOTA: incorpora procesar el csv obtenido (de todas las actividades y todas las páginas al ejecutar) a un csv depurado entendible por humanos, y después, procesarlo todo a números para Machine Learning

# Obtención detalles actividades, incluida imágen (Web Scrapping)
(web Ayuntamiento de Madrid)

[Enlace portal web Ayuntamiento de Madrid Actividades Infantiles](https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD)

<img src="../img/AyuntMadrid_actividades_02.png" width="700">

## Recursividad: obtener datos de todas las actividades

Resumen de código con funciones para todas actividades y páginas. Depurado y ordenado.

In [ ]:
# 01 importación de librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re
import time


In [ ]:
# 02 Función extraer_info_por_actividad:Encapsulado de mi código de extracción en una función
def extraer_info_por_actividad(url_actividad, indice=0, total=0):
    """
    Extrae información detallada de una actividad a partir de su URL.
    
    Args:
        url_actividad (str): URL de la página de detalle de la actividad
        indice (int, optional): Índice de la actividad. Por defecto 0.
        total (int, optional): Total de actividades. Por defecto 0.
        
    Returns:
        dict or None: Diccionario con los datos de la actividad, o None si hay error
    """
    if indice > 0 and total > 0:
        print(f"\nProcesando actividad {indice} de {total}...")
    
    print(f"Accediendo a la página de detalle: {url_actividad}")
    
    # Hacer una solicitud a la página de detalle con un pequeño retraso para no sobrecargar el servidor
    time.sleep(2)  # Esperar 2 segundos entre solicitudes
    response_detalle = requests.get(url_actividad)

    if response_detalle.status_code == 200:
        soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
        
        # Diccionario para almacenar los datos de la actividad
        datos_actividad = {}

        contenedor_actividad = soup_detalle.find('div', class_='Panel 1.1')
        if not contenedor_actividad:
            print("No se encontró el contenedor de contenido de la actividad.")
            return None

        tramites_content = contenedor_actividad.find('div', class_='tramites-content')
        tiny_text = contenedor_actividad.find('div', class_='tiny-text')
        actividades_info = contenedor_actividad.find('div', class_='actividades-info')
        info_actividad_fecha_lugar = contenedor_actividad.find('div', class_='info-actividad')

        # Título
        titulo_elem = soup_detalle.find('h3', class_='summary-title')
        datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"

        # Descripción
        datos_actividad['descripción'] = tiny_text.p.text.strip() if tiny_text and tiny_text.p else "Sin descripción"

        # Edad
        datos_actividad['edad'] = "No especificada"

        # Función auxiliar para extraer el rango de edad de un texto
        def extraer_rango_edad(texto):
            # Patrón para "de X a Y años"
            patron_rango = re.search(r'de\s+(\d+)\s+a\s+(\d+)\s+años', texto, re.IGNORECASE)
            if patron_rango:
                return f"de {patron_rango.group(1)} a {patron_rango.group(2)} años"
            
            # Patrón para "Edad: X y Y años" o similares con dos puntos
            if ':' in texto:
                partes = texto.split(':', 1)
                if 'años' in partes[1]:
                    # Extraer solo la parte que contiene la edad después de los dos puntos
                    edad_parte = partes[1].strip()
                    # Si hay texto adicional después de "años", cortarlo
                    if ' años' in edad_parte:
                        edad_parte = edad_parte[:edad_parte.find(' años') + 5]
                    return edad_parte
            
            # Patrón para "entre X y Y años"
            patron_entre = re.search(r'entre\s+(\d+)\s+y\s+(\d+)\s+años', texto, re.IGNORECASE)
            if patron_entre:
                return f"entre {patron_entre.group(1)} y {patron_entre.group(2)} años"

            # Patrón para "X y Y años" (normalmente consecutivos)
            patron_y = re.search(r'(\d+)\s+y\s+(\d+)\s+años', texto, re.IGNORECASE)
            if patron_y:
                return f"{patron_y.group(1)} y {patron_y.group(2)} años"
                                        
            # Patrón para "a partir de X años"
            patron_a_partir = re.search(r'a\s+partir\s+de\s+(\d+)\s+años', texto, re.IGNORECASE)
            if patron_a_partir:
                return f"a partir de {patron_a_partir.group(1)} años"
            
            # Si contiene años y alguna palabra clave pero no encaja en patrones anteriores
            if 'años' in texto and any(palabra in texto.lower() for palabra in ['edad', 'niñ', 'de', 'a partir']):
                return texto.strip()
            
            return None

        # Buscar en blockquote dentro de tiny_text
        if tiny_text:
            for blockquote in tiny_text.find_all('blockquote'):
                if 'Edad:' in blockquote.text and 'años' in blockquote.text:
                    edad_extraida = extraer_rango_edad(blockquote.text)
                    if edad_extraida:
                        datos_actividad['edad'] = edad_extraida
                        break
            
            # Si aún no se ha encontrado, buscar en todo el texto de tiny_text
            if datos_actividad['edad'] == "No especificada":
                posibles_frases = re.split(r'[.,;:()\n]', tiny_text.get_text())
                for frase in posibles_frases:
                    edad_extraida = extraer_rango_edad(frase)
                    if edad_extraida:
                        datos_actividad['edad'] = edad_extraida
                        break

        # Buscar en tramites_content si aún no se ha encontrado
        if datos_actividad['edad'] == "No especificada" and tramites_content:
            posibles_frases = re.split(r'[.,;:()\n]', tramites_content.get_text())
            for frase in posibles_frases:
                edad_extraida = extraer_rango_edad(frase)
                if edad_extraida:
                    datos_actividad['edad'] = edad_extraida
                    break

        # Inscripción
        texto_inscripcion = ""
        inscripcion_parrafo = ""

        # Primero buscar en div.info-actividad si existe:
        if info_actividad_fecha_lugar:
            h4 = info_actividad_fecha_lugar.find('h4', class_='inscripcion')
            if h4:
                siguiente_p = h4.find_next('p', class_='text-date')
                if siguiente_p:
                    inscripcion_parrafo = siguiente_p.text.strip()

        # Después buscar también en tiny_text (independientemente de si ya encontramos algo)
        if tiny_text:
            match = re.search(r'([^.]*inscripción[^.]*\.)', tiny_text.get_text(), re.IGNORECASE)
            if match:
                texto_inscripcion = match.group(0).strip()
            
            # Además, buscar en párrafos específicos dentro de tiny_text
            for p in tiny_text.find_all('p'):
                # Verificar si contiene la palabra "inscr" en cualquier parte del párrafo
                if re.search(r'inscr', p.text, re.IGNORECASE):
                    # Si ya tenemos contenido de info_actividad, concatenamos
                    if texto_inscripcion:
                        texto_inscripcion += ". " + p.text.strip()
                    else:
                        texto_inscripcion = p.text.strip()
                    break

        # Asignar el valor final de inscripción según lo que se haya encontrado
        if inscripcion_parrafo or texto_inscripcion:
            # Si tenemos ambos contenidos, los concatenamos
            if inscripcion_parrafo and texto_inscripcion:
                datos_actividad['inscripción'] = inscripcion_parrafo + ". " + texto_inscripcion
            elif inscripcion_parrafo:
                datos_actividad['inscripción'] = inscripcion_parrafo
            else:
                datos_actividad['inscripción'] = texto_inscripcion
        else:
            datos_actividad['inscripción'] = "No especificada"

        # Periodicidad
        datos_actividad['periodicidad'] = "No especificada"
        if tiny_text:
            contenido = tiny_text.get_text()
            dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
            periodos = {'semana': 'semanal', 'mes': 'mensual', 'año': 'anual'}
            for dia in dias_semana:
                for periodo, valor in periodos.items():
                    if re.search(rf'un\s+{dia}\s+(al|cada)\s+{periodo}', contenido, re.IGNORECASE):
                        datos_actividad['periodicidad'] = valor
                        break
                if datos_actividad['periodicidad'] != "No especificada":
                    break
            if datos_actividad['periodicidad'] == "No especificada":
                patrones = ['diaria', 'semanal', 'quincenal', 'mensual']
                for patron in patrones:
                    if re.search(patron, contenido, re.IGNORECASE):
                        datos_actividad['periodicidad'] = patron
                        break

        # Día/días
        datos_actividad['día_días'] = "No especificado"
        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
        dias_encontrados = []
        
        # Primero buscar en tiny_text
        if tiny_text:
            contenido = tiny_text.get_text() 
            dias_encontrados = [dia for dia in dias_semana if re.search(dia, contenido, re.IGNORECASE)]
            
        # Si no hay resultados, buscar también en info_actividad_fecha_lugar
        if not dias_encontrados and info_actividad_fecha_lugar:
            contenido_info = info_actividad_fecha_lugar.get_text()
            dias_encontrados = [dia for dia in dias_semana if re.search(dia, contenido_info, re.IGNORECASE)]

        # Asignar el resultado    
        datos_actividad['día_días'] = ", ".join(dias_encontrados) if dias_encontrados else "No especificado"

        # Extraer horario
        def extraer_horario(texto_html, buscar_bloque=False):
            soup = BeautifulSoup(str(texto_html), 'html.parser')
            bloque = ""

            if buscar_bloque:
                # Buscar el tag que contenga la palabra "Horarios:"
                tag_horarios = None
                for tag in soup.find_all(string=re.compile(r'\bHorarios:\b', re.IGNORECASE)):
                    tag_horarios = tag.find_parent()
                    if tag_horarios:
                        siguiente_ul = None
                        for sibling in tag_horarios.find_all_next():
                            if sibling.name == 'ul':
                                siguiente_ul = sibling
                                break
                        if siguiente_ul:
                            items = [li.get_text(" ", strip=True) for li in siguiente_ul.find_all("li")]
                            bloque = " ".join(items)
                            bloque = re.sub(r'\s+', ' ', bloque).strip()
                            if bloque:
                                return bloque

            # Si no encontramos un bloque o no se busca, buscar patrones en texto plano como respaldo
            texto = soup.get_text(separator=' ', strip=True)

            # Para "de XX:XX a YY:YY horas" o "de XX:XX a YY:YY h" o "de XX:XX a YY:YY h."
            patron_rango = re.search(r'(de\s+\d{1,2}[:.]\d{2}\s+a\s+\d{1,2}[:.]\d{2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
            if patron_rango:
                return patron_rango.group(1).strip()

            # Para "de XX a YY horas" o "de XX a YY h" o "de XX a YY h."
            patron_rango_simple = re.search(r'(de\s+\d{1,2}\s+a\s+\d{1,2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
            if patron_rango_simple:
                return patron_rango_simple.group(1).strip()

            # Para "a las XX[:YY] horas" o "a las XX[:YY] h" o "a las XX[:YY] h."
            patron_a_las = re.search(r'(a\s+las\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
            if patron_a_las:
                return patron_a_las.group(1).strip()

            # Para "de XX a YY horas y de ZZ:ZZ a WW:WW horas" (o versiones con "h"/"h.")
            patron_multiples = re.search(r'(de\s+\d{1,2}(?:[:.]\d{2})?\s+a\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b(?:\s+y\s+de\s+\d{1,2}(?:[:.]\d{2})?\s+a\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b)?)', texto, re.IGNORECASE)
            if patron_multiples:
                return patron_multiples.group(1).strip()

            # Para fechas específicas seguidas de horario: "5, 11, 12... de abril de 10:30 a 13:30 h."
            patron_fecha_hora = re.search(r'(\d+(?:,\s+\d+)*(?:\s+y\s+\d+)?\s+de\s+[a-zá-úñ]+\s+de\s+\d{1,2}[:.]\d{2}\s+a\s+\d{1,2}[:.]\d{2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
            if patron_fecha_hora:
                return patron_fecha_hora.group(1).strip()

            # Por si acaso, si contiene "horas"/"h"/"h." y algún formato de hora pero no encaja en los patrones anteriores
            if re.search(r'\b(?:horas|h\.?)\b', texto.lower()) and re.search(r'\d{1,2}(?:[:.]\d{2})?', texto):
                return texto.strip()

            return None

        # Inicializar el campo horario
        datos_actividad['horario'] = "No especificado"

        # Buscar en info_actividad_fecha_lugar
        if info_actividad_fecha_lugar:
            fragmentos = re.split(r'[.,;()\n]', info_actividad_fecha_lugar.get_text())
            for frag in fragmentos:
                horario_extraido = extraer_horario(frag)
                if horario_extraido:
                    datos_actividad['horario'] = horario_extraido
                    break

        # Buscar en tiny_text si aún no se ha encontrado
        if datos_actividad['horario'] == "No especificado" and tiny_text:
            texto_tiny = tiny_text.get_text()
            if re.search(r'\bHorarios:\b', texto_tiny, re.IGNORECASE):
                horario_extraido = extraer_horario(tiny_text, buscar_bloque=True)
                if horario_extraido:
                    datos_actividad['horario'] = horario_extraido
            else:
                fragmentos = re.split(r'[.,;()\n]', texto_tiny)
                for frag in fragmentos:
                    horario_extraido = extraer_horario(frag)
                    if horario_extraido:
                        datos_actividad['horario'] = horario_extraido
                        break

        # Fecha
        datos_actividad['fecha'] = "Sin fecha"
        if info_actividad_fecha_lugar:
            # Buscar primero el encabezado h4 con clase "fecha title9"
            fecha_header = info_actividad_fecha_lugar.find('h4', class_='fecha title9')
            if fecha_header:
                # Si lo encuentra, buscar el p.text-date inmediatamente después de este encabezado
                fecha_elem = fecha_header.find_next('p', class_='text-date')
                if fecha_elem:
                    fecha_texto = fecha_elem.text.strip()
                    # Eliminar la parte de la hora si existe
                    if ' a las ' in fecha_texto:
                        fecha_texto = fecha_texto.split(' a las ')[0].strip()
                    datos_actividad['fecha'] = fecha_texto
                else:
                    # Método 2: Si no encuentra text-date, buscar div.tiny-text después del encabezado
                    tiny_text_div = fecha_header.find_next('div', class_='tiny-text')
                    if tiny_text_div:
                        # Extraer el texto del primer párrafo en tiny-text que generalmente contiene la fecha
                        fecha_parrafo = tiny_text_div.find('p')
                        if fecha_parrafo:
                            fecha_texto = fecha_parrafo.text.strip()
                            # Eliminar la parte de la hora si existe
                            if ' a las ' in fecha_texto:
                                fecha_texto = fecha_texto.split(' a las ')[0].strip()
                            datos_actividad['fecha'] = fecha_texto

        # Lugar nombre
        datos_actividad['lugar_nombre'] = "Sin lugar"
        if info_actividad_fecha_lugar:
            lugar_elem = info_actividad_fecha_lugar.find('a', class_='url fn')
            if lugar_elem:
                datos_actividad['lugar_nombre'] = lugar_elem.text.strip()

        # Dirección
        datos_actividad['lugar_dirección'] = "Sin dirección"
        direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
        if direccion_elem and direccion_elem.find('dd'):
            direccion = direccion_elem.find('dd').text.strip()
            datos_actividad['lugar_dirección'] = re.sub(r'\s+', ' ', direccion)

        # Precio y recomendación
        datos_actividad['precio'] = "No especificado"
        datos_actividad['recomendación'] = "No especificado"
        if actividades_info:
            precio_elem = actividades_info.find('p', class_='gratuita')
            if precio_elem:
                datos_actividad['precio'] = precio_elem.text.strip()
            recomendacion_elem = actividades_info.find('p', class_='ninos')
            if recomendacion_elem:
                datos_actividad['recomendación'] = recomendacion_elem.text.strip()
        if datos_actividad['precio'] == "No especificado" and tiny_text:
            precio_parrafos = []
            # Opción 2: O tomar solo párrafos dentro de blockquotes
            blockquotes = tiny_text.find_all('blockquote')
            for bq in blockquotes:
                for p in bq.find_all('p'):
                    if 'euro' in p.text.lower():
                        precio_parrafos.append(p.text.strip())
    
            if precio_parrafos:
                datos_actividad['precio'] = ". ".join(precio_parrafos)

        # URL ampliar información
        datos_actividad['url_ampliar_info'] = "No disponible"
        if tramites_content:
            h4 = tramites_content.find('h4', class_='title8', string='Amplíe información')
            if h4:
                p = h4.find_next('p')
                if p and p.find('a'):
                    url_info = p.find('a')['href']
                    datos_actividad['url_ampliar_info'] = "https://www.madrid.es" + url_info if not url_info.startswith('http') else url_info
        
        # Extraer URL de imagen embebida
        if tramites_content:
            imagen_elem = tramites_content.find('img')
            if imagen_elem and imagen_elem.get('src'):
                url_imagen = imagen_elem['src']
                if not url_imagen.startswith('http'):
                    url_imagen = "https://www.madrid.es" + url_imagen
                datos_actividad['url_imagen'] = url_imagen
            else:
                datos_actividad['url_imagen'] = "No disponible"
        else:
            datos_actividad['url_imagen'] = "No disponible"


        # Añadir URL de la página de detalle para referencia
        datos_actividad['url_actividad'] = url_actividad
        
        # Imprimir los resultados en el formato exacto solicitado
        print("\nDatos obtenidos de la actividad:")
        for clave, valor in datos_actividad.items():
            if clave != 'url_actividad':  # No mostrar la url_actividad para mantener el formato original
                print(f"{clave}: {valor}")
        
        return datos_actividad
    else:
        print(f"Error al acceder a la página de detalle: código {response_detalle.status_code}")
        return None

In [ ]:
# 03 Función para procesar las actividades de una pagina (de cada pagina, la se vaya iterando)
def procesar_pagina_actividades(url_pagina, numero_pagina, total_paginas):
    """
    Procesa una página de actividades y devuelve los datos de cada actividad.
    
    Args:
        url_pagina (str): URL de la página a procesar
        numero_pagina (int): Número de página actual
        total_paginas (int): Número total de páginas
    
    Returns:
        list: Lista de diccionarios con los datos de las actividades
    """
    print(f"\nAccediendo a la página {numero_pagina+1}/{total_paginas}: {url_pagina}")
    actividades_pagina = []
    
    try:
        # Pequeña pausa para no sobrecargar el servidor
        time.sleep(1)
        response_pagina = requests.get(url_pagina)
        
        if response_pagina.status_code == 200:
            soup_pagina = BeautifulSoup(response_pagina.text, 'html.parser')
            contenedor_actividades = soup_pagina.find('ul', class_='events-results docs')
            
            if contenedor_actividades:
                # Buscar todas las actividades dentro del contenedor
                actividades = contenedor_actividades.find_all('div', class_='event-info')
                print(f"Encontradas {len(actividades)} actividades en la página {numero_pagina+1}")
                
                # Iterar sobre todas las actividades encontradas
                for indice, actividad in enumerate(actividades, 1):
                    # Extraer el enlace de la actividad
                    elemento_enlace = actividad.find('a', class_='event-link')
                    if elemento_enlace and 'href' in elemento_enlace.attrs:
                        url_actividad = elemento_enlace['href']
                        if not url_actividad.startswith('http'):
                            url_actividad = "https://www.madrid.es" + url_actividad
                        
                        # Llamar a la función para extraer la información de la actividad
                        datos_actividad = extraer_info_por_actividad(url_actividad, indice, len(actividades))
                        
                        # Si se encontraron datos, añadirlos a la lista
                        if datos_actividad:
                            actividades_pagina.append(datos_actividad)
                    else:
                        print("No se encontró el enlace a la página de detalle.")
            else:
                print(f"No se encontró el contenedor de actividades en la página {numero_pagina+1}")
        else:
            print(f"Error al acceder a la página {numero_pagina+1}: código {response_pagina.status_code}")
    except Exception as e:
        print(f"Error inesperado al procesar la página {numero_pagina+1}: {str(e)}")
    
    return actividades_pagina

In [ ]:
# 04 Función obtener cuales paginas, una o todas, y cuales actividades, rango o todas, que el usuario quiere extraer
def procesar_rango_actividades(todas_actividades, inicio, fin):
    """
    Devuelve un subconjunto de actividades basado en índices de inicio y fin.
    
    Args:
        todas_actividades (list): Lista completa de actividades
        inicio (int): Índice de la primera actividad a incluir
        fin (int): Índice de la última actividad a incluir (inclusive)
    
    Returns:
        list: Subconjunto de actividades
    """
    # Validar índices
    if inicio < 1:
        inicio = 1
    if fin > len(todas_actividades):
        fin = len(todas_actividades)
    
    print(f"Filtrando actividades: se guardarán solo desde índice {inicio} hasta {fin} (de {len(todas_actividades)} totales)")
    
    
    print(f"Procesando actividades en el rango {inicio} a {fin} de {len(todas_actividades)}")
    return todas_actividades[inicio-1:fin]


In [ ]:
df

In [ ]:
# 05 CELDA PRINCIPAL PARA EJECUTAR Y LLAMAR A LAS FUNCIONES ANIDADAS

# URL base
url_base = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD"

# Obtener fecha actual
fecha_hoy = date.today().strftime("%d-%m-%Y")

# Primera solicitud para obtener el total
response = requests.get(url_base)
if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extraer el número total de actividades
    contenedor_principal = soup.find('ul', id='totalResultsUL')
    total_actividades = 0
    if contenedor_principal:
        total_elementos = contenedor_principal.find('strong')
        if total_elementos:
            total_actividades = int(total_elementos.text)
            print(f"Consulta fecha {fecha_hoy}. Número total de actividades: {total_actividades}")
        else:
            print("No se encontró el elemento que contiene el número total")
    else:
        print("No se encontró el contenedor de resultados totales")
    
    # Calcular número total de páginas
    if total_actividades % 25 == 0:
        total_paginas = total_actividades // 25
    else:
        total_paginas = (total_actividades // 25) + 1
    print(f"Número total de páginas a recorrer: {total_paginas}")
    print("Nota: La página 1 corresponde al valor page=0 de la web")
    
    # Preguntar qué páginas procesar
    while True:
        pagina_input = input(f"\nIntroduce la página a consultar (1-{total_paginas}) o 't' para todas: ")
        
        if pagina_input.lower() == 't':
            pagina_inicio = 0
            pagina_fin = total_paginas
            print(f"Se procesarán todas las páginas (1 a {total_paginas})")
            break
        else:
            try:
                pagina_num = int(pagina_input)-1
                if 0 <= pagina_num < total_paginas:
                    pagina_inicio = pagina_num
                    pagina_fin = pagina_num + 1
                    print(f"Se procesará solo la página {pagina_num +1}")
                    break
                else:
                    print(f"Error: El número debe estar entre 1 y {total_paginas}")
            except ValueError:
                print("Error: Introduce un número válido o 't'")
    
    # Preguntar si se quiere procesar un rango específico de actividades
    procesar_rango = input("\nDentro de esta página, ¿deseas procesar solo un rango específico de actividades? (entre 1 y 25) (s/n): ")
    inicio_actividad = 0
    fin_actividad = None  # Valor None indica procesar todas
    
    if procesar_rango.lower() == 's':
        try:
            inicio_actividad = int(input("Introduce el índice de la primera actividad a procesar: "))
            fin_actividad = int(input("Introduce el índice de la última actividad a procesar (Enter para todas): "))

            if fin_actividad <= 25:  
                print(f"Se procesarán actividades desde índice {inicio_actividad} hasta {fin_actividad}")
            else:
                print(f"Se procesarán todas las actividades desde el índice {inicio_actividad}")
            
        except ValueError:
            print("Valores no válidos. Se procesarán todas las actividades.")
            inicio_actividad = 1
            fin_actividad = None
    
    # Lista para almacenar todas las actividades
    todas_actividades = []
    
    # Recorrer las páginas seleccionadas
    for pagina in range(pagina_inicio, pagina_fin):
        url_pagina = f"{url_base}&page={pagina}"
        actividades_pagina = procesar_pagina_actividades(url_pagina, pagina, total_paginas)
        todas_actividades.extend(actividades_pagina)
    
    # Aplicar el filtro de rango si es necesario
    actividades_a_guardar = todas_actividades
    if fin_actividad is not None:
        actividades_a_guardar = procesar_rango_actividades(todas_actividades, inicio_actividad, fin_actividad)
    
    # Crear DataFrame con las actividades seleccionadas y guardar CSV
    if actividades_a_guardar:
        df = pd.DataFrame(actividades_a_guardar)
        directorio = '../data/raw/'
        # Comprobar si el directorio existe, si no, crearlo
        os.makedirs(directorio, exist_ok=True)
        
        # Preparar nombre del archivo según el formato solicitado
        if pagina_inicio == 0 and pagina_fin == total_paginas:
            paginas_str = "todas"
        else:
            paginas_str = str(pagina_fin - pagina_inicio)
            
        if fin_actividad is not None:
            act_str = f"act{inicio_actividad+1}-{fin_actividad+1}"
        else:
            act_str = f"act{len(actividades_a_guardar)}"
            
        nombre_archivo = f'{directorio}actividades_detalle_página{paginas_str}_{act_str}_{fecha_hoy}.csv'
        
        df.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
        print(f"\nDatos guardados correctamente en '{nombre_archivo}'")
        print(f"Se han extraído datos de {len(actividades_a_guardar)} actividades.")
    else:
        print("\nNo se pudieron extraer datos de ninguna actividad.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

In [ ]:
df.sample(8)

## Trabajando en el csv completo obtenido

Ahora voy a desglosar algunos datos y organizarlos hasta lograr un Dataframe más manejable y fácil de visualizar. Para ello voy a abrir el archivo obtenido antes

### Carga e inspección básica del DataFrame de partida

In [2]:
# seleccionar la fecha del csv
import os
import pandas as pd
# Definir ruta del archivo
directorio = '../data/raw/'
nombre_archivo_trabajo = 'actividades_detalle_páginatodas_act261_02-05-2025.csv'
ruta_archivo_trabajo = os.path.join(directorio, nombre_archivo_trabajo)

df = pd.read_csv(ruta_archivo_trabajo)

In [ ]:
df[título].if ['Fiesta de pijamas'] isin df['título']

In [49]:
df[df['título'].str.contains('Fiesta de pijamas', na=False)]

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,precio,recomendación,url_ampliar_info,url_imagen,url_actividad,fecha_inicio,fecha_fin,fecha_inicio_year,fecha_inicio_month,fecha_inicio_day,fecha_inicio_dayofweek,fecha_fin_year,fecha_fin_month,fecha_fin_day,fecha_fin_dayofweek,duracion_dias,hora_inicio,duracion_horas,dia_lunes,dia_martes,dia_miércoles,dia_jueves,dia_viernes,dia_sábado,dia_domingo,precio_gratuito,distrito,distrito_Arganzuela,distrito_Barajas,distrito_Carabanchel,distrito_Centro,distrito_Chamartín,distrito_Chamberí,distrito_Ciudad Lineal,distrito_Fuencarral-El Pardo,distrito_Hortaleza,distrito_Latina,distrito_Moncloa-Aravaca,distrito_Moratalaz,distrito_No especificado,distrito_Puente de Vallecas,distrito_Retiro,distrito_Salamanca,distrito_San Blas-Canillejas,distrito_Tetuán,distrito_Usera,distrito_Vicálvaro,distrito_Villa de Vallecas,distrito_Villaverde,requiere_inscripcion,recomendado_ninos,tiene_url_info,ciudad_madrid


In [54]:
df[df['periodicidad'].isin(['No especificada'])]
# mal extraída. Sería necesario tomar mejor día_días en el scrapping, y así obtener mejor la periodicidad.
# además, ver también en la columna 'fecha' para ver si aparecen dos días diferentes o iguales.
TODO Realizar mejor extracción de día_días, y con ello, de periodicidad

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,precio,recomendación,url_ampliar_info,url_imagen,url_actividad,fecha_inicio,fecha_fin,fecha_inicio_year,fecha_inicio_month,fecha_inicio_day,fecha_inicio_dayofweek,fecha_fin_year,fecha_fin_month,fecha_fin_day,fecha_fin_dayofweek,duracion_dias,hora_inicio,duracion_horas,dia_lunes,dia_martes,dia_miércoles,dia_jueves,dia_viernes,dia_sábado,dia_domingo,precio_gratuito,distrito,distrito_Arganzuela,distrito_Barajas,distrito_Carabanchel,distrito_Centro,distrito_Chamartín,distrito_Chamberí,distrito_Ciudad Lineal,distrito_Fuencarral-El Pardo,distrito_Hortaleza,distrito_Latina,distrito_Moncloa-Aravaca,distrito_Moratalaz,distrito_No especificado,distrito_Puente de Vallecas,distrito_Retiro,distrito_Salamanca,distrito_San Blas-Canillejas,distrito_Tetuán,distrito_Usera,distrito_Vicálvaro,distrito_Villa de Vallecas,distrito_Villaverde,requiere_inscripcion,recomendado_ninos,tiene_url_info,ciudad_madrid
0,Talleres creativos en idioma serbio,Talleres creativos para niños y niñas de 3 a16...,Talleres creativos para niños y niñas de 3 a16...,No especificada,No especificada,domingo,de 11 a 13 horas,Del domingo 22 de septiembre de 2024 al doming...,Biblioteca Pública Municipal Iván de Vargas (C...,"CALLE SAN JUSTO, 5 28005 MADRID",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/UnidadesDescentralizadas...,https://www.madrid.es/portales/munimadrid/es/I...,2024-09-22,2025-06-22,2024.0,9.0,22.0,6.0,2025.0,6.0,22.0,6.0,274.0,11.0,2.0,0,0,0,0,0,0,1,1,Centro,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,1
7,Ludoteca en Centro Sociocultural El Greco,Viernes de 17:30 a 20:30 horas y sábados de 10...,de 5 a 12 años,Más información sobre inscripciones y activida...,No especificada,"viernes, sábado",de 17:30 a 20:30 horas,Del viernes 4 de octubre de 2024 al sábado 28 ...,Centro Sociocultural El Greco (Latina),"CALLE EL GRECO, 29 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,No disponible,https://www.madrid.es/portales/munimadrid/es/I...,2024-10-04,2025-06-28,2024.0,10.0,4.0,4.0,2025.0,6.0,28.0,5.0,268.0,17.5,3.0,0,0,0,0,1,1,0,1,Latina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1
8,Ludoteca en Centro Cultural José Luis Sampedro,Viernes de 17:30 a 20:30 horas y sábados de 10...,de 5 a 12 años,Para más información sobre inscripciones y act...,No especificada,"viernes, sábado",de 17:30 a 20:30 horas,Del viernes 4 de octubre de 2024 al sábado 28 ...,Centro Cultural José Luis Sampedro (Latina),"CALLE CASTROSERNA, 4 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,No disponible,https://www.madrid.es/portales/munimadrid/es/I...,2024-10-04,2025-06-28,2024.0,10.0,4.0,4.0,2025.0,6.0,28.0,5.0,268.0,17.5,3.0,0,0,0,0,1,1,0,1,Latina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1
9,Ludoteca en Centro Sociocultural Almirante Chu...,Viernes de 17:30 a 20:30 horas y sábados de 10...,de 5 a 12 años,Más información sobre inscripciones y activida...,No especificada,"viernes, sábado",de 17:30 a 20:30 horas,Del viernes 4 de octubre de 2024 al sábado 28 ...,Centro Sociocultural Almirante Churruca (Latina),"CALLE FUENTE DEL TIRO, 72 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,No disponible,https://www.madrid.es/portales/munimadrid/es/I...,2024-10-04,2025-06-28,2024.0,10.0,4.0,4.0,2025.0,6.0,28.0,5.0,268.0,17.5,3.0,0,0,0,0,1,1,0,1,Latina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1
10,Ludoteca en Centro Cultural Fernando de los Ríos,Viernes de 17:30 a 20:30 horas y sábados de 10...,de 5 a 12 años,Más información sobre inscripciones y activida...,No especificada,"viernes, sábado",de 17:30 a 20:30 horas,Del vierne

In [48]:
df.head(2)

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,precio,recomendación,url_ampliar_info,url_imagen,url_actividad,fecha_inicio,fecha_fin,fecha_inicio_year,fecha_inicio_month,fecha_inicio_day,fecha_inicio_dayofweek,fecha_fin_year,fecha_fin_month,fecha_fin_day,fecha_fin_dayofweek,duracion_dias,hora_inicio,duracion_horas,dia_lunes,dia_martes,dia_miércoles,dia_jueves,dia_viernes,dia_sábado,dia_domingo,precio_gratuito,distrito,distrito_Arganzuela,distrito_Barajas,distrito_Carabanchel,distrito_Centro,distrito_Chamartín,distrito_Chamberí,distrito_Ciudad Lineal,distrito_Fuencarral-El Pardo,distrito_Hortaleza,distrito_Latina,distrito_Moncloa-Aravaca,distrito_Moratalaz,distrito_No especificado,distrito_Puente de Vallecas,distrito_Retiro,distrito_Salamanca,distrito_San Blas-Canillejas,distrito_Tetuán,distrito_Usera,distrito_Vicálvaro,distrito_Villa de Vallecas,distrito_Villaverde,requiere_inscripcion,recomendado_ninos,tiene_url_info,ciudad_madrid
0,Talleres creativos en idioma serbio,Talleres creativos para niños y niñas de 3 a16...,Talleres creativos para niños y niñas de 3 a16...,No especificada,No especificada,domingo,de 11 a 13 horas,Del domingo 22 de septiembre de 2024 al doming...,Biblioteca Pública Municipal Iván de Vargas (C...,"CALLE SAN JUSTO, 5 28005 MADRID",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/UnidadesDescentralizadas...,https://www.madrid.es/portales/munimadrid/es/I...,2024-09-22,2025-06-22,2024.0,9.0,22.0,6.0,2025.0,6.0,22.0,6.0,274.0,11.0,2.0,0,0,0,0,0,0,1,1,Centro,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,1
1,Tertulias en Inglés (infantil) Biblioteca Vall...,Sin descripción,de 8 a 12 años,Para participar es necesario realizar inscripc...,semanal,miércoles,de 18 a 19:30 horas,Del miércoles 25 de septiembre de 2024 al miér...,Biblioteca Pública Municipal Vallecas (Puente ...,"CALLE PUERTO DEL MONASTERIO, 1 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,No disponible,https://www.madrid.es/portales/munimadrid/es/I...,2024-09-25,2025-06-11,2024.0,9.0,25.0,2.0,2025.0,6.0,11.0,2.0,260.0,18.0,1.5,0,0,1,0,0,0,0,1,Puente de Vallecas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 261 entries, 0 to 260
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   título            261 non-null    object
 1   descripción       237 non-null    object
 2   edad              261 non-null    object
 3   inscripción       261 non-null    object
 4   periodicidad      261 non-null    object
 5   día_días          261 non-null    object
 6   horario           261 non-null    object
 7   fecha             261 non-null    object
 8   lugar_nombre      261 non-null    object
 9   lugar_dirección   261 non-null    object
 10  precio            261 non-null    object
 11  recomendación     261 non-null    object
 12  url_ampliar_info  261 non-null    object
 13  url_imagen        261 non-null    object
 14  url_actividad     261 non-null    object
dtypes: object(15)
memory usage: 30.7+ KB


### Categorizar: Definir categorías y subcategorías

Función desarrollada para obtener la subcategoría y categorias (estas pueden actualizarse) de cada evento en las columnas "título" y "descripción", aunque estas pueden cambiarse y/o adaptarse a otro dataframe.

In [5]:
# Función para extraer categorías y subcategorías, seleccionando columnas donde se busca
import pandas as pd
import re

def extraer_categorias_subcategorias(df):
    """
    Extrae categorías y subcategorías basadas en palabras clave dentro de las columnas 'título' y 'descripción'.
    
    Parámetros:
    -----------
    df : DataFrame
        El DataFrame donde se realizará la búsqueda
        
    Retorna:
    --------
    DataFrame con las columnas 'categoría' y 'subcategoría' añadidas
    """
    # Paso 1: Crear una copia del DataFrame para no modificar el original
    df_resultado = df.copy()
    
    # Paso 2: Definir las columnas donde buscar palabras clave
    columnas_busqueda = ["título", "descripción"]
    
    # Paso 3: Definir categorías principales
    categorias = [
        "La Ciudad", "Cultura y Espectáculos", "Educación", "Aire Libre", "Deporte",
    ]

    # Paso 4: Definir subcategorías para cada categoría
    subcategorias = {
        "La Ciudad": ["Actividades", "Carnaval", "Semana Santa", "Verano", "Centros de Atención a Las Familias", "Navidad"],
        "Cultura y Espectáculos": ["Cuentacuentos", "Títeres y marionetas", "Magia", "Club de lectura", "Cine Infantil", 
                                  "Circo", "Conciertos", "Exposiciones", "Monumentos", "Museos", "Proyecciones", 
                                  "Talleres", "Teatro Infantil", "Música", "Bailes"],
        "Educación": ["Ayudas y Becas", "Campamentos Urbanos", "Ludotecas", "Idiomas"],
        "Aire Libre": ["Actividades al Aire Libre", "Actividades con animales", "Parques y jardines"],
        "Deporte": ["Deportes de aventura", "Deportes de invierno", "Deportes acuáticos", "Deportes de cancha", 
                    "Rocódromos", "Deporte en la calle", "Carreras y eventos"]
    }

    # Paso 5: Definir palabras clave para cada subcategoría
    palabras_clave_subcategorias = {
    # Subcategoría: La Ciudad
    "Actividades": ["Evento", "Programa", "Actividad general", "Jornada", "Sesión"],
    "Carnaval": ["Carnaval", "Desfile", "Disfraces", "Fiesta de Carnaval"],
    "Semana Santa": ["Semana Santa", "Procesión", "Pascua", "Domingo de Ramos"],
    "Verano": ["Verano", "Vacaciones", "Programa estival", "Escuela de verano"],
    "Centros de Atención a Las Familias": ["CAF", "Centro de Atención Familiar", "Atención a familias", "Apoyo familiar"],
    "Navidad": ["Navidad", "Reyes Magos", "Belén", "Fiesta navideña", "Villancicos"],

    # Subcategoría: Cultura y Espectáculos
    "Cuentacuentos": ["Narración oral", "Historias para niños", "Cuentos infantiles"],
    "Títeres y marionetas": ["Marionetas", "Teatro de títeres", "Títeres de guante"],
    "Magia": ["Magia", "Ilusionismo", "Trucos de magia", "Magos infantiles"],
    "Club de lectura": ["Lectura", "Cuentos", "Libros infantiles", "Club de lectura"],
    "Cine Infantil": ["Películas infantiles", "Cine para niños", "Cine en familia"],
    "Circo": ["Circo", "Malabares", "Equilibristas", "Trapecistas", "Payaso" , "Payasos"],
    "Conciertos": ["Concierto", "Música en vivo", "Actuación musical"],
    "Exposiciones": ["Exposición", "Muestra", "Galería", "Arte infantil"],
    "Monumentos": ["Visita guiada", "Monumento histórico", "Ruta patrimonial"],
    "Museos": ["Museo", "Visita museo", "Museo infantil"],
    "Proyecciones": ["Proyección", "Audiovisual", "Documental", "Corto infantil"],
    "Talleres": ["Taller", "Talleres", "Manualidades", "Arte infantil", "Actividad práctica"],
    "Teatro Infantil": ["Teatro", "Obra infantil", "Espectáculo teatral", "Compañía"],
    "Música": ["Música", "Músicas", "Musical", "Musicales", "Concierto", "Opera"],
    "Bailes": ["Danza", "Coreografía", "Baile infantil", "Zumba para niños"],

    # Subcategoría: Educación
    "Ayudas y Becas": ["Beca", "Ayuda educativa", "Subvención", "Apoyo económico"],
    "Campamentos Urbanos": ["Campamento", "Escuela urbana", "Colonias urbanas"],
    "Ludotecas": ["Ludoteca", "Juego libre", "Centro lúdico"],
    "Idiomas": ["Inglés", "Francés","Alemán", "Italiano", "Idioma", "Idiomas", "Clases de idiomas"],

    # Subcategoría: Aire Libre
    "Actividades al Aire Libre": ["Aire libre", "Excursión", "Actividad exterior", "Picnic"],
    "Actividades con animales": ["Animales", "Granja escuela", "Visita zoológico", "Taller con animales"],
    "Parques y jardines": ["Parque", "Jardín", "Espacio verde", "Parque infantil"],

    # Subcategoría: Deporte
    "Deportes de aventura": ["Escalada", "Ciclismo de montaña", "Trekking", "Parkour", "Buceo", "Barranquismo", "Rafting", "Parapente", "Alpinismo", "Rápel"],
    "Deportes de invierno": ["Esquí", "Snowboard", "Patinaje sobre hielo", "Ski alpino", "Esquí de fondo"],
    "Deportes acuáticos": ["Surf", "Natación", "Kitesurf", "Windsurf", "Buceo", "Paddle surf", "Canoeing"],
    "Deportes de cancha": ["Fútbol", "Baloncesto", "Tenis", "Voleibol", "Bádminton", "Padel"],
    "Rocódromos": ["Escalada en roca", "Escalada indoor", "Boulder", "Escalada deportiva"],
    "Deporte en la calle": ["Skateboard", "Patinaje", "Parkour", "Roller", "Bicicross"],
    "Carreras y eventos": ["Maratón", "Carrera popular", "Crossfit", "Triatlón", "Carrera de obstáculos"]
}
    
    # Paso 6: Crear un diccionario que mapee cada subcategoría a su categoría principal
    subcategoria_a_categoria = {}
    for categoria, lista_subcats in subcategorias.items():
        for subcat in lista_subcats:
            subcategoria_a_categoria[subcat] = categoria
    
    # Paso 7: Inicializar nuevas columnas con valores por defecto
    df_resultado["categoría"] = "No especificada"
    df_resultado["subcategoría"] = "No especificada"
    
    # Paso 8: Procesar cada fila del DataFrame
    for indice, fila in df_resultado.iterrows():
        # Inicializar texto completo para búsqueda
        texto_completo = ""
        
        # Extraer texto de las columnas de búsqueda
        for columna in columnas_busqueda:
            if columna in df_resultado.columns and pd.notna(fila.get(columna, "")):
                texto_completo += str(fila[columna]).lower() + " "
        
        # Conjuntos para almacenar categorías y subcategorías encontradas
        subcategorias_encontradas = set()
        categorias_encontradas = set()
        
        # Buscar palabras clave en el texto
        for subcategoria, palabras in palabras_clave_subcategorias.items():
            for palabra in palabras:
                # Buscar la palabra completa (con límites de palabra)
                if re.search(r'\b' + re.escape(palabra.lower()) + r'\b', texto_completo):
                    subcategorias_encontradas.add(subcategoria)
                    
                    # Obtener la categoría de esta subcategoría
                    if subcategoria in subcategoria_a_categoria:
                        categorias_encontradas.add(subcategoria_a_categoria[subcategoria])
        
        # Asignar resultados a las columnas
        if subcategorias_encontradas:
            df_resultado.at[indice, 'subcategoría'] = '. '.join(sorted(subcategorias_encontradas))
        
        if categorias_encontradas:
            df_resultado.at[indice, 'categoría'] = '. '.join(sorted(categorias_encontradas))
    
    # Paso 9: Retornar el DataFrame con las nuevas columnas
    return df_resultado

### Transformación en DF depurado con datos limpios

incorporada la extracción de categoría y subcategoría

In [6]:
# Código para depurar df y poder visualizar si datos son correctos, con columna original y extracción
# incorporada la extracción de categoría y subcategoría
import pandas as pd
import numpy as np
from datetime import datetime
import re

# Hacer una copia del dataframe original
df_depurado = df.copy()

# Aplicar la función para extraer categorías y subcategorías
df_depurado = extraer_categorias_subcategorias(df_depurado)

# Verificar que las columnas se han creado correctamente
print(f"Columnas originales/iniciales del DataFrame: {df_depurado.columns.tolist()}")
print(f"Número de categorías únicas: {df_depurado['categoría'].nunique()}")
print(f"Número de subcategorías únicas: {df_depurado['subcategoría'].nunique()}")

# Reordenar columnas para que categoría y subcategoría aparezcan después de descripción
if 'descripción' in df_depurado.columns:
    # Obtener todas las columnas actuales
    columnas = df_depurado.columns.tolist()
    
    # Encontrar la posición de la columna 'descripción'
    indice_descripcion = columnas.index('descripción')
    
    # Crear nueva lista de columnas reordenadas
    columnas_reordenadas = (
        # Columnas antes de 'descripción'
        columnas[:indice_descripcion+1] + 
        # Insertar 'categoría' y 'subcategoría'
        ['categoría', 'subcategoría'] + 
        # Resto de columnas, excluyendo 'categoría' y 'subcategoría' si ya estaban en otra posición
        [col for col in columnas[indice_descripcion+1:] if col not in ['categoría', 'subcategoría']]
    )
    
    # Aplicar el nuevo orden de columnas
    df_depurado = df_depurado[columnas_reordenadas]

# 1. Procesamiento de edad mejorado
# ---------------------
def extract_age_range(age_text):
    if pd.isna(age_text):
        return np.nan, np.nan
    
    # Convertir a texto en minúsculas para facilitar los patrones
    text = str(age_text).lower()
    
    # Lista para almacenar todas las edades encontradas
    ages = []
    
    # 1. Patrón: "X y Y años", "de X a Y años" o "entre X y Y años" o "desde los X años"
    range_patterns = [
    r'\s*(\d+)\s*y\s*(\d+)\s*años',
    r'de\s*(\d+)\s*a\s*(\d+)\s*años',
    r'entre\s*(\d+)\s*y\s*(\d+)\s*años',
    r'desde\s*(\d+)\s*hasta\s*(\d+)\s*años',
    r'desde\s*los\s*(\d+)\s*años\s*hasta\s*los\s*(\d+)',
    r'desde\s*los\s*(\d+)\s*hasta\s*los\s*(\d+)\s*años',
]
    
    for pattern in range_patterns:
        matches = re.search(pattern, text)
        if matches:
            min_age = int(matches.group(1))
            max_age = int(matches.group(2))
            ages.extend([min_age, max_age])
            break
    
    # 2. Patrón: "a partir de X años"
    start_patterns = [
    r'a\s*partir\s*de\s*(\d+)\s*años',
    r'a\s*partir\s*(\d+)\s*años',
    r'para\s*mayores\s*de\s*(\d+)\s*años',
    r'mayores\s*de\s*(\d+)\s*años',
    r'desde\s*los\s*(\d+)\s*años',
    r'(\d+)\s*y\s*(\d+)\s*años',
    ]
    
    for pattern in start_patterns:
        matches = re.search(pattern, text)
        if matches and not ages:  # Solo si no encontramos un rango antes
            min_age = int(matches.group(1))
            max_age = 18  # Límite superior de 18 años
            ages.extend([min_age, max_age])
            break
    
    # 3. Patrón: "hasta X años" o "menores de X años"
    end_patterns = [
        r'hasta (\d+) años',
        r'menores de (\d+) años'
    ]
    
    for pattern in end_patterns:
        matches = re.search(pattern, text)
        if matches and not ages:  # Solo si no encontramos otro patrón antes
            min_age = 0  # Asumimos desde 0 años
            max_age = int(matches.group(1))
            ages.extend([min_age, max_age])
            break
    
    # 4. Patrón: "X años" (edad específica)
    single_age = re.search(r'(\d+) años', text)
    if single_age and not ages:  # Solo si no encontramos otro patrón antes
        age = int(single_age.group(1))
        if age <= 18:  # Solo procesar si es menor o igual a 18
            ages.extend([age, age])
    
    # 5. Patrón: "de X meses" (convertir meses a años)
    months_pattern = re.search(r'(\d+) meses', text)
    if months_pattern and not ages:
        months = int(months_pattern.group(1))
        years = months // 12
        if years <= 18:
            ages.extend([years, years])
    
    # 6. Patrón: "de 0 meses a X años" o variantes
    mixed_pattern = re.search(r'de 0 meses a (\d+) años', text)
    if mixed_pattern and not ages:
        max_age = int(mixed_pattern.group(1))
        if max_age <= 18:
            ages.extend([0, max_age])
    
    # 7. Caso especial para "nacidos entre años"
    birth_years = re.findall(r'nacidos entre los años (\d{4}) y (\d{4})', text)
    if birth_years and not ages:
        current_year = datetime.now().year
        birth_min = int(birth_years[0][0])
        birth_max = int(birth_years[0][1])
        age_max = current_year - birth_min
        age_min = current_year - birth_max
        if age_min <= 18:  # Solo consideramos si al menos parte del rango es menor o igual a 18
            age_min = max(0, age_min)
            age_max = min(18, age_max)
            ages.extend([age_min, age_max])
       
    # Si encontramos edades, procesarlas
    if ages:
        min_age = int(min(ages))
        max_age = int(max(ages))
        
        # Limitar edades a 18 años como máximo
        min_age = min(min_age, 18)
        max_age = min(max_age, 18)
        
        return min_age, max_age
    
    # Si no se encontró ningún patrón válido
    return np.nan, np.nan

# Aplicar la función y crear columnas para edad mínima y máxima
edad_min_temp, edad_max_temp = zip(*df_depurado['edad'].apply(extract_age_range))

# Convertir a enteros (si es posible)
# df_depurado['edad_min'] = pd.Series(edad_min_temp).apply(lambda x: int(x) if pd.notna(x) else np.nan)
# df_depurado['edad_max'] = pd.Series(edad_max_temp).apply(lambda x: int(x) if pd.notna(x) else np.nan)

# Asignar directamente a las columnas, usando el tipo 'Int64' de pandas
df_depurado['edad_min'] = pd.Series(edad_min_temp, dtype='Int64')
df_depurado['edad_max'] = pd.Series(edad_max_temp, dtype='Int64')

# 2. Procesar inscripción
# ----------------------
def extract_inscription_required(inscription_text):
    if pd.isna(inscription_text):
        return np.nan
    
    if 'No especificada' in inscription_text:
        return np.nan
    
    # Si tiene algún contenido que no sea "No especificada", asumimos que requiere inscripción
    return True

df_depurado['requiere_inscripcion'] = df_depurado['inscripción'].apply(extract_inscription_required)

# 3. Procesar periodicidad (mantener como está o NaN)
# ---------------------------------------------------
df_depurado['periodicidad'] = df_depurado['periodicidad'].replace('No especificada', np.nan)

# 4. Procesamiento de días de la semana
# -------------------------------------
# Mantener los días como están (o NaN)
df_depurado['día_días'] = df_depurado['día_días'].replace('No especificado', np.nan)

# 5. Procesar horario
# ------------------
def extract_time_range(horario_text):
    if pd.isna(horario_text):
        return np.nan, np.nan
    
    # Convertir a minúsculas para facilitar patrones
    text = str(horario_text).lower()
    
    # Patrón para "de X:XX a Y:YY horas"
    range_time_pattern = r'de (\d+)(?::(\d+))? a (\d+)(?::(\d+))? horas'
    
    # Patrón para "a las X horas"
    single_time_pattern = r'a las (\d+)(?::(\d+))? horas'
    
    # Intentar matchear un rango de horas
    range_match = re.search(range_time_pattern, text)
    if range_match:
        groups = range_match.groups()
        start_hour = int(groups[0])
        start_min = int(groups[1]) if groups[1] else 0
        end_hour = int(groups[2])
        end_min = int(groups[3]) if groups[3] else 0
        
        start_time = f"{start_hour:02d}:{start_min:02d}"
        end_time = f"{end_hour:02d}:{end_min:02d}"
        
        return start_time, end_time
    
    # Intentar matchear una hora única
    single_match = re.search(single_time_pattern, text)
    if single_match:
        groups = single_match.groups()
        hour = int(groups[0])
        minute = int(groups[1]) if groups[1] else 0
        
        start_time = f"{hour:02d}:{minute:02d}"
        return start_time, np.nan
    
    # Otro patrón: "Horario: X:XX horas"
    other_pattern = re.search(r'horario:\s*(\d+)(?::(\d+))?\s*horas', text)
    if other_pattern:
        groups = other_pattern.groups()
        hour = int(groups[0])
        minute = int(groups[1]) if groups[1] else 0
        
        start_time = f"{hour:02d}:{minute:02d}"
        return start_time, np.nan
    
    return np.nan, np.nan

# Aplicar la función y crear columnas para hora inicio y fin
df_depurado['hora_inicio'], df_depurado['hora_fin'] = zip(*df_depurado['horario'].apply(extract_time_range))

# 6. Procesar fechas
# -----------------
def extract_date_range(fecha_text):
    if pd.isna(fecha_text):
        return np.nan, np.nan
    
    text = str(fecha_text)
    
    # Patrón para fechas como "Del lunes 23 de junio de 2025 al jueves 31 de julio de 2025"
    range_date_pattern = r'Del .+ (\d+) de (\w+) de (\d{4}) al .+ (\d+) de (\w+) de (\d{4})'
    
    # Patrón para una sola fecha como "Sábado 26 de abril de 2025"
    single_date_pattern = r'(\w+) (\d+) de (\w+) de (\d{4})'
    
    # Mapeo de meses en español a números
    month_map = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
        'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }
    
    # Intentar matchear un rango de fechas
    range_match = re.search(range_date_pattern, text)
    if range_match:
        start_day, start_month_name, start_year, end_day, end_month_name, end_year = range_match.groups()
        start_date = f"{int(start_day):02d}/{month_map.get(start_month_name.lower(), 1):02d}/{start_year}"
        end_date = f"{int(end_day):02d}/{month_map.get(end_month_name.lower(), 1):02d}/{end_year}"
        return start_date, end_date
    
    # Intentar matchear una fecha única
    single_match = re.search(single_date_pattern, text)
    if single_match:
        _, day, month_name, year = single_match.groups()
        date_str = f"{int(day):02d}/{month_map.get(month_name.lower(), 1):02d}/{year}"
        return date_str, date_str
    
    return np.nan, np.nan

# Aplicar la función y crear columnas para fecha inicio y fin
df_depurado['fecha_inicio'], df_depurado['fecha_fin'] = zip(*df_depurado['fecha'].apply(extract_date_range))

# 7. Lugar (mantener como está)
# ----------------------------
# El lugar_nombre y lugar_dirección se mantienen como están

# 8. Extraer distrito
# -----------------

# Añadir columna ciudad
df_depurado['ciudad']= 'Madrid'

# Lista completa de distritos de Madrid
distritos_madrid = [
    'Centro', 'Arganzuela', 'Retiro', 'Salamanca', 'Chamartín', 'Tetuán', 'Chamberí',
    'Fuencarral-El Pardo', 'Moncloa-Aravaca', 'Latina', 'Carabanchel', 'Usera',
    'Puente de Vallecas', 'Moratalaz', 'Ciudad Lineal', 'Hortaleza', 'Villaverde',
    'Villa de Vallecas', 'Vicálvaro', 'San Blas-Canillejas', 'Barajas'
]

def extract_district(row):
    # Buscar distrito en paréntesis en el nombre del lugar
    if pd.notna(row['lugar_nombre']):
        district_match = re.search(r'\(([^)]+)\)', row['lugar_nombre'])
        if district_match:
            district = district_match.group(1)
            # Verificar si es un distrito conocido o contiene uno
            for d in distritos_madrid:
                if d in district:
                    return d
    
    # Buscar en lugar_nombre sin paréntesis
    if pd.notna(row['lugar_nombre']):
        for distrito in distritos_madrid:
            if distrito in row['lugar_nombre']:
                return distrito
    
    # Buscar en título
    if pd.notna(row['título']):
        for distrito in distritos_madrid:
            if distrito in row['título']:
                return distrito
    
    # Buscar en descripción
    if pd.notna(row['descripción']):
        for distrito in distritos_madrid:
            if distrito in row['descripción']:
                return distrito
    
    # Si no se encuentra ningún distrito
    return np.nan

# Aplicar la función para extraer distrito
df_depurado['distrito'] = df_depurado.apply(extract_district, axis=1)

# 9. Procesar precio
# ----------------
def extract_price(price_text):
    if pd.isna(price_text):
        return np.nan
    
    if 'Gratuito' in price_text:
        return 0
    
    # Intentar extraer un precio numérico
    price_match = re.search(r'(\d+(?:,\d+)?)', price_text)
    if price_match:
        price_str = price_match.group(1).replace(',', '.')
        try:
            return float(price_str)
        except ValueError:
            pass
    
    return np.nan

df_depurado['precio_valor'] = df_depurado['precio'].apply(extract_price)

# 10. Procesar recomendación
# -----------------------
def extract_recommendation(rec_text):
    if pd.isna(rec_text) or rec_text == 'No disponible':
        return False
    
    return True

df_depurado['recomendado'] = df_depurado['recomendación'].apply(extract_recommendation)

# 11. Procesar URL ampliar info
# ---------------------------
def has_url_info(url_text):
    if pd.isna(url_text) or url_text == 'No disponible':
        return False
    
    return True

df_depurado['tiene_url_info'] = df_depurado['url_ampliar_info'].apply(has_url_info)

# 12. Procesar URL imagen
# ---------------------------
# def has_url_imagen(url_imagen):
#     if pd.isna(url_imagen) or url_imagen == 'No disponible':
#         return False
    
#     return True

df_depurado['tiene_url_imagen'] = df_depurado['url_imagen'].apply(has_url_info)


# Reorganizar las columnas para mantener un orden similar al original
# Primero obtenemos las columnas originales
columnas_originales = df.columns.tolist()

# Luego definimos las nuevas columnas en el orden que queremos que aparezcan junto a sus originales
nuevas_columnas_ordenadas = []
for col in columnas_originales:
    nuevas_columnas_ordenadas.append(col)  # Columna original
    
    # Agregamos las columnas derivadas correspondientes
    if col == 'descripción':
        nuevas_columnas_ordenadas.extend(['categoría', 'subcategoría'])
    if col == 'edad':
        nuevas_columnas_ordenadas.extend(['edad_min', 'edad_max'])
    elif col == 'inscripción':
        nuevas_columnas_ordenadas.append('requiere_inscripcion')
    elif col == 'horario':
        nuevas_columnas_ordenadas.extend(['hora_inicio', 'hora_fin'])
    elif col == 'fecha':
        nuevas_columnas_ordenadas.extend(['fecha_inicio', 'fecha_fin'])
    elif col == 'lugar_dirección':
        nuevas_columnas_ordenadas.extend(['ciudad', 'distrito'])
    elif col == 'precio':
        nuevas_columnas_ordenadas.append('precio_valor')
    elif col == 'recomendación':
        nuevas_columnas_ordenadas.append('recomendado')
    elif col == 'url_ampliar_info':
        nuevas_columnas_ordenadas.append('tiene_url_info')
    elif col == 'url_imagen':
        nuevas_columnas_ordenadas.append('tiene_url_imagen')

# Filtrar solo las columnas que existen en el dataframe
nuevas_columnas_ordenadas = [col for col in nuevas_columnas_ordenadas if col in df_depurado.columns]

# Reordenar el dataframe
df_depurado = df_depurado[nuevas_columnas_ordenadas]

# Visualizar el resultado
print(f"Dimensiones del dataframe depurado: {df_depurado.shape}")
print(f"Columnas en orden: {df_depurado.columns.tolist()}")

# Mostrar algunas filas para verificar
#print(df_depurado.head())

# Mostrar estadísticas descriptivas para las nuevas columnas numéricas
# numeric_cols = ['edad_min', 'edad_max', 'precio_valor']
# print(df_depurado[numeric_cols].describe())

Columnas originales/iniciales del DataFrame: ['título', 'descripción', 'edad', 'inscripción', 'periodicidad', 'día_días', 'horario', 'fecha', 'lugar_nombre', 'lugar_dirección', 'precio', 'recomendación', 'url_ampliar_info', 'url_imagen', 'url_actividad', 'categoría', 'subcategoría']
Número de categorías únicas: 10
Número de subcategorías únicas: 43
Dimensiones del dataframe depurado: (261, 30)
Columnas en orden: ['título', 'descripción', 'categoría', 'subcategoría', 'edad', 'edad_min', 'edad_max', 'inscripción', 'requiere_inscripcion', 'periodicidad', 'día_días', 'horario', 'hora_inicio', 'hora_fin', 'fecha', 'fecha_inicio', 'fecha_fin', 'lugar_nombre', 'lugar_dirección', 'ciudad', 'distrito', 'precio', 'precio_valor', 'recomendación', 'recomendado', 'url_ampliar_info', 'tiene_url_info', 'url_imagen', 'tiene_url_imagen', 'url_actividad']


In [7]:
df_depurado['precio_valor'].unique()

array([ 0., nan, 10.])

In [43]:
df_depurado['periodicidad'].unique()

array([nan, 'semanal', 'quincenal', 'mensual'], dtype=object)

In [8]:
df_depurado[df_depurado['precio_valor'] == 10.0]


,título,descripción,categoría,subcategoría,edad,edad_min,edad_max,inscripción,requiere_inscripcion,periodicidad,...,distrito,precio,precio_valor,recomendación,recomendado,url_ampliar_info,tiene_url_info,url_imagen,tiene_url_imagen,url_actividad
28,Talleres del Price (5 y 6 años),¡Diviértete aprendiendo mientras descubres tod...,Cultura y Espectáculos,Talleres,5 y 6 años,5,6,No especificada,NaN,NaN,...,NaN,Abono trimestral (10 sesiones): 140 euros. Abo...,10.0,No especificado,True,No disponible,False,https://www.madrid.es/UnidadWeb/UGBBDD/MadridD...,True,https://www.madrid.es/portales/munimadrid/es/I...


In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


In [10]:
df_depurado.head(1)

,título,descripción,categoría,subcategoría,edad,edad_min,edad_max,inscripción,requiere_inscripcion,periodicidad,día_días,horario,hora_inicio,hora_fin,fecha,fecha_inicio,fecha_fin,lugar_nombre,lugar_dirección,ciudad,distrito,precio,precio_valor,recomendación,recomendado,url_ampliar_info,tiene_url_info,url_imagen,tiene_url_imagen,url_actividad
0,Talleres creativos en idioma serbio,"Talleres creativos para niños y niñas de 3 a16 años en idioma serbio (manualidades, bellas artes, ciencia, música, literatura, teatro, baile, cine, cultura).",Cultura y Espectáculos. Educación,Idiomas. Música. Talleres. Teatro Infantil,Talleres creativos para niños y niñas de 3 a16 años en idioma serbio,3,16,No especificada,NaN,NaN,domingo,de 11 a 13 horas,11:00,13:00,Del domingo 22 de septiembre de 2024 al domingo 22 de junio de 2025,22/09/2024,22/06/2025,Biblioteca Pública Municipal Iván de Vargas (Centro),"CALLE SAN JUSTO, 5 28005 MADRID",Madrid,Centro,Gratuito,0.0,Recomendado para niñas y niños,True,No disponible,False,https://www.madrid.es/UnidadesDescentralizadas/Bibliotecas/BibliotecasPublicas/Actividades/Actividades_Adultos/Clubes_Lectura/ficheros/SERBIA_260X260.jpg,True,https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Talleres-creativos-en-idioma-serbio/?vgnextfmt=default&vgnextoid=394cdaa14d721910VgnVCM2000001f4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD


In [19]:
df_depurado.sample(2)

,título,descripción,categoría,subcategoría,edad,edad_min,edad_max,inscripción,requiere_inscripcion,periodicidad,...,distrito,precio,precio_valor,recomendación,recomendado,url_ampliar_info,tiene_url_info,url_imagen,tiene_url_imagen,url_actividad
42,La magia existe,"EL mago Héctor, de la escuela de Ana Tamariz, ...",Cultura y Espectáculos,Magia,No especificada,<NA>,<NA>,No especificada,NaN,NaN,...,NaN,Gratuito,0.0,Recomendado para niñas y niños,True,https://www.madrid.es/portales/munimadrid/es/I...,True,https://www.madrid.es/UnidadWeb/UGBBDD/Activid...,True,https://www.madrid.es/portales/munimadrid/es/I...
39,Érase una vez un pato - Teatro La Proa (Cuba),La noticia sobre la existencia de una extraña...,Aire Libre. Cultura y Espectáculos,Actividades con animales. Teatro Infantil,a partir de 6 años,6,18,No especificada,NaN,NaN,...,Retiro,Gratuito,0.0,Recomendado para niñas y niños,True,No disponible,False,https://www.madrid.es/UnidadWeb/UGBBDD/Activid...,True,https://www.madrid.es/portales/munimadrid/es/I...


In [ ]:
df_depurado.head(1)['edad']

0    Talleres creativos para niños y niñas de 3 a16 años en idioma serbio
Name: edad, dtype: object

In [18]:
pd.reset_option('display.max_colwidth')
pd.reset_option('display.max_columns')

Paso discreccional para guardar el dataframe para incorporar en la página web y establecer menus desplegables

In [20]:
# para guardar para Web Full Stack
directorio = '../data/processed/forWebFullStack/'
# Comprobar si el directorio existe, si no, crearlo
os.makedirs(directorio, exist_ok=True)
  
nombre_archivo = f'{directorio}actividades_detalles_limpiasWeb_02mayo2025.csv'

df_depurado.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
print(f"\nDatos guardados correctamente en '{nombre_archivo}'")



Datos guardados correctamente en '../data/processed/forWebFullStack/actividades_detalles_limpiasWeb_02mayo2025.csv'


### Limpieza adicional* columnas ya procesadas (información redundante)
*opcional

In [21]:
# CÓdigo ejecución OPCIONAL: para eliminar las columnas ya procesadas, y dejar el df mejor visualizable

columnas_a_eliminar = [
    'edad', 'inscripción', 'horario', 'fecha', 
    'lugar_nombre', 'lugar_dirección', 'precio', 'recomendación',
    'url_imagen', 'url_ampliar_info', 'url_actividad',
]
   
# Verificar qué columnas existen antes de eliminarlas
cols_to_drop = [col for col in columnas_a_eliminar if col in df_depurado.columns]
df_depurado2 = df_depurado.drop(columns=cols_to_drop)
df_depurado2

,título,descripción,categoría,subcategoría,edad_min,edad_max,requiere_inscripcion,periodicidad,día_días,hora_inicio,hora_fin,fecha_inicio,fecha_fin,ciudad,distrito,precio_valor,recomendado,tiene_url_info,tiene_url_imagen
0,Talleres creativos en idioma serbio,Talleres creativos para niños y niñas de 3 a16...,Cultura y Espectáculos. Educación,Idiomas. Música. Talleres. Teatro Infantil,3,16,NaN,NaN,domingo,11:00,13:00,22/09/2024,22/06/2025,Madrid,Centro,0.0,True,False,True
1,Tertulias en Inglés (infantil) Biblioteca Vall...,Sin descripción,Educación,Idiomas,8,12,True,semanal,miércoles,18:00,19:30,25/09/2024,11/06/2025,Madrid,Puente de Vallecas,0.0,True,True,False
2,Club de lectura (infantil) Biblioteca David Gi...,"Una de nuestras actividades favoritas, dónde l...",Cultura y Espectáculos,Club de lectura,9,12,True,quincenal,lunes,18:00,19:30,30/09/2024,09/06/2025,Madrid,Salamanca,0.0,True,True,True
3,Taller de cómic (Infantil) Biblioteca Pozo del...,Son talleres de creación dirigidos a dar a con...,Cultura y Espectáculos,Talleres,8,11,True,quincenal,lunes,17:30,19:00,30/09/2024,02/06/2025,Madrid,Puente de Vallecas,0.0,True,True,True
4,Club de lectura (infantil) Biblioteca San Fermín,"Una de nuestras actividades favoritas, dónde l...",Cultura y Espectáculos,Club de lectura,9,12,True,quincenal,miércoles,18:00,19:30,02/10/2024,18/06/2025,Madrid,Usera,0.0,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,Campamentos de verano 2025 para menores nacido...,Sin descripción,La Ciudad,Verano,3,12,NaN,NaN,"lunes, viernes",08:30,15:30,23/06/2025,29/08/2025,Madrid,NaN,0.0,True,True,False
257,Verde que te quiero verde - Zaguán Teatro (Cas...,"En 1923, un joven Federico García Lorca decidi...",Cultura y Espectáculos,Teatro Infantil,<NA>,<NA>,NaN,NaN,"sábado, domingo",18:30,NaN,28/06/2025,29/06/2025,Madrid,Retiro,0.0,True,False,True
258,Los más pequeños también plantamos,NaN,No especificada,No especificada,<NA>,<NA>,NaN,NaN,domingo,11:00,NaN,29/06/2025,29/06/2025,Madrid,Centro,0.0,True,True,False
259,Campamento urbano de verano en Usera,Campamento para niños y niñas donde se realiza...,Educación. La Ciudad,Campamentos Urbanos. Verano,3,18,True,NaN,"martes, viernes",07:30,09:00,01/07/2025,05/09/2025,Madrid,Usera,0.0,True,False,True


In [26]:
# para guardar para poder retomarlo directamente en la limpieza sin ejecutar todas las celdas de arriba
directorio = '../data/processed/'
# Comprobar si el directorio existe, si no, crearlo
os.makedirs(directorio, exist_ok=True)
  
nombre_archivo = f'{directorio}actividades(261)_detalles_depurado2_02mayo2025.csv'

df_depurado2.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
print(f"\nDatos guardados correctamente en '{nombre_archivo}'")



Datos guardados correctamente en '../data/processed/actividades(261)_detalles_depurado2_02mayo2025.csv'


In [42]:
df_depurado2['periodicidad'].unique()

array([nan, 'semanal', 'quincenal', 'mensual'], dtype=object)

### Transformación final en DF listo para ML
(todo a valores numéricos)

__________________________________

In [91]:
# Versión 1 preparación para Machine Learning
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from datetime import datetime

def df_to_ml(df_original, max_tfidf_features=5):
    """
    Prepara un DataFrame para el entrenamiento de un modelo de recomendación KNN.
    
    Parámetros:
    -----------
    df_original : DataFrame
        El DataFrame original que contiene la información de las actividades.
    max_tfidf_features : int, opcional (default=5)
        Número máximo de características a extraer con TF-IDF.
        
    Retorna:
    --------
    DataFrame preparado para machine learning con todas las características numéricas.
    """
    # Crear una copia del DataFrame original para no modificarlo
    df_ml = df_original.copy()
    
    # 1. Limpieza básica de datos
    # ---------------------------
    df_ml.replace(["No especificada", "No especificado", "No disponible"], np.nan, inplace=True)
    
    # 2. Procesamiento de texto con TF-IDF
    # -----------------------------------
    # Extraer longitud del título y descripción
    df_ml['titulo_longitud'] = df_ml['título'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
    df_ml['descripcion_longitud'] = df_ml['descripción'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
    
    # Definir stop words en español
    stop_words = ['de', 'la', 'el', 'en', 'y', 'a', 'para', 'con', 'por', 'del', 
                  'los', 'las', 'un', 'una', 'al', 'es', 'su', 'se', 'que', 'no']
    
    # TF-IDF para título
    if df_ml['título'].notna().sum() > 1:
        tfidf_titulo = TfidfVectorizer(
            max_features=max_tfidf_features, 
            stop_words=stop_words,
            ngram_range=(1, 2)  # Incluir bigramas para capturar frases clave
        )
        titulo_tfidf = tfidf_titulo.fit_transform(df_ml['título'].fillna(''))
        titulo_tfidf_df = pd.DataFrame(
            titulo_tfidf.toarray(), 
            columns=[f'titulo_tfidf_{i}' for i in range(titulo_tfidf.shape[1])]
        )
        df_ml = pd.concat([df_ml, titulo_tfidf_df], axis=1)
    
    # TF-IDF para descripción
    if df_ml['descripción'].notna().sum() > 1:
        tfidf_desc = TfidfVectorizer(
            max_features=max_tfidf_features, 
            stop_words=stop_words,
            ngram_range=(1, 2)
        )
        desc_tfidf = tfidf_desc.fit_transform(df_ml['descripción'].fillna(''))
        desc_tfidf_df = pd.DataFrame(
            desc_tfidf.toarray(), 
            columns=[f'desc_tfidf_{i}' for i in range(desc_tfidf.shape[1])]
        )
        df_ml = pd.concat([df_ml, desc_tfidf_df], axis=1)
    
    # 3. Procesamiento de columnas categóricas
    # --------------------------------------
    
    # Procesar categoría y subcategoría con OneHotEncoder
    categorical_cols = ['categoría', 'subcategoría']
    for col in categorical_cols:
        if col in df_ml.columns:
            # Convertir a string para asegurar compatibilidad
            df_ml[col] = df_ml[col].astype(str)
            
            # Manejar múltiples categorías (separadas por puntos)
            # Expandir columnas múltiples en columnas individuales
            categories = set()
            for val in df_ml[col].dropna():
                for category in val.split('. '):
                    categories.add(category)
            
            # Crear columnas binarias para cada categoría
            for category in categories:
                if category and category != 'nan':
                    col_name = f"{col}_{category}"
                    df_ml[col_name] = df_ml[col].apply(
                        lambda x: 1 if pd.notna(x) and category in x else 0
                    )
    
    # 4. Procesar edad (si no está ya procesada)
    # ----------------------------------------
    if 'edad_min' in df_ml.columns and 'edad_max' in df_ml.columns:
        # Rellenar valores faltantes con valores por defecto
        df_ml['edad_min'] = pd.to_numeric(df_ml['edad_min'], errors='coerce').fillna(0)
        df_ml['edad_max'] = pd.to_numeric(df_ml['edad_max'], errors='coerce').fillna(18)
        
        # Crear rangos de edad como características
        age_ranges = [(0, 3), (4, 6), (7, 9), (10, 12), (13, 15), (16, 18)]
        for min_age, max_age in age_ranges:
            col_name = f"edad_{min_age}_{max_age}"
            df_ml[col_name] = (
                ((df_ml['edad_min'] <= max_age) & (df_ml['edad_max'] >= min_age)) | 
                ((df_ml['edad_min'] <= min_age) & (df_ml['edad_max'] >= max_age))
            ).astype(int)
    
    # 5. Procesar fechas
    # ----------------
    date_cols = ['fecha_inicio', 'fecha_fin']
    for col in date_cols:
        if col in df_ml.columns:
            # Convertir a datetime si no lo está ya
            if not pd.api.types.is_datetime64_any_dtype(df_ml[col]):
                df_ml[col] = pd.to_datetime(df_ml[col], errors='coerce', dayfirst=True)
            
            if df_ml[col].notna().any():
                # Extraer componentes de fecha
                df_ml[f'{col}_year'] = df_ml[col].dt.year
                df_ml[f'{col}_month'] = df_ml[col].dt.month
                df_ml[f'{col}_day'] = df_ml[col].dt.day
                df_ml[f'{col}_dayofweek'] = df_ml[col].dt.dayofweek
                
                # Características estacionales
                df_ml[f'{col}_is_summer'] = ((df_ml[col].dt.month >= 6) & 
                                             (df_ml[col].dt.month <= 9)).astype(int)
                df_ml[f'{col}_is_winter'] = ((df_ml[col].dt.month == 12) | 
                                             (df_ml[col].dt.month <= 2)).astype(int)
                df_ml[f'{col}_is_weekend'] = (df_ml[col].dt.dayofweek >= 5).astype(int)
    
    # Calcular duración del evento en días
    if 'fecha_inicio' in df_ml.columns and 'fecha_fin' in df_ml.columns:
        if pd.api.types.is_datetime64_any_dtype(df_ml['fecha_inicio']) and pd.api.types.is_datetime64_any_dtype(df_ml['fecha_fin']):
            df_ml['duracion_dias'] = (df_ml['fecha_fin'] - df_ml['fecha_inicio']).dt.days + 1
            df_ml['duracion_dias'] = df_ml['duracion_dias'].fillna(1)  # Asumimos 1 día para eventos sin rango
            
            # Codificar duración en rangos
            df_ml['duracion_corta'] = (df_ml['duracion_dias'] <= 1).astype(int)
            df_ml['duracion_media'] = ((df_ml['duracion_dias'] > 1) & 
                                      (df_ml['duracion_dias'] <= 7)).astype(int)
            df_ml['duracion_larga'] = (df_ml['duracion_dias'] > 7).astype(int)
    
    # 6. Procesar horas
    # ---------------
    time_cols = ['hora_inicio', 'hora_fin']
    for col in time_cols:
        if col in df_ml.columns:
            # Convertir a formato numérico si es string
            if df_ml[col].dtype == 'object':
                df_ml[col] = pd.to_datetime(df_ml[col], format='%H:%M', errors='coerce').dt.hour + \
                             pd.to_datetime(df_ml[col], format='%H:%M', errors='coerce').dt.minute / 60
            
            # Características de periodo del día
            if pd.notna(df_ml[col]).any():
                df_ml[f'{col}_manana'] = ((df_ml[col] >= 7) & (df_ml[col] < 12)).astype(int)
                df_ml[f'{col}_mediodia'] = ((df_ml[col] >= 12) & (df_ml[col] < 15)).astype(int)
                df_ml[f'{col}_tarde'] = ((df_ml[col] >= 15) & (df_ml[col] < 19)).astype(int)
                df_ml[f'{col}_noche'] = ((df_ml[col] >= 19) | (df_ml[col] < 7)).astype(int)
    
    # 7. Procesar días de la semana
    # --------------------------
    if 'día_días' in df_ml.columns:
        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
        for dia in dias_semana:
            df_ml[f'dia_{dia}'] = df_ml['día_días'].str.contains(dia, case=False, na=False).astype(int)
        
        # Agregar características de fin de semana y entre semana
        df_ml['dia_findesemana'] = ((df_ml['dia_sábado'] == 1) | (df_ml['dia_domingo'] == 1)).astype(int)
        df_ml['dia_entresemana'] = ((df_ml['dia_lunes'] == 1) | (df_ml['dia_martes'] == 1) | 
                                   (df_ml['dia_miércoles'] == 1) | (df_ml['dia_jueves'] == 1) | 
                                   (df_ml['dia_viernes'] == 1)).astype(int)
    
     # 8. Procesar periodicidad
    # ---------------------
    if 'periodicidad' in df_ml.columns:
        df_ml['periodicidad'] = df_ml['periodicidad'].fillna('Ninguna')
        df_ml['periodicidad_diaria'] = (df_ml['periodicidad'] == 'diaria').astype(int)
        df_ml['periodicidad_semanal'] = (df_ml['periodicidad'] == 'semanal').astype(int)
        df_ml['periodicidad_quincenal'] = (df_ml['periodicidad'] == 'quincenal').astype(int)
        df_ml['periodicidad_mensual'] = (df_ml['periodicidad'] == 'mensual').astype(int)
    
    
    # 9. Procesar distritos
    # ------------------
    if 'distrito' in df_ml.columns:
        df_ml['distrito'] = df_ml['distrito'].fillna('No especificado')
        distritos = df_ml['distrito'].unique()
        
        distrito_cols = {
            f'distrito_{d}': (df_ml['distrito'] == d).astype(int) 
            for d in distritos if d and d != 'nan'
        }
    
        df_ml = pd.concat([df_ml, pd.DataFrame(distrito_cols, index=df_ml.index)], axis=1)

    
    # 10. Procesar otras características booleanas
    # ---------------------------------------
    # Convertir columnas booleanas a numéricas
    bool_cols = ['requiere_inscripcion', 'recomendado', 'tiene_url_info', 'tiene_url_imagen']
    for col in bool_cols:
        if col in df_ml.columns:
            # Asegurarse de que pandas infiera el tipo de la columna antes de trabajar con ella
            # df_ml[col] = df_ml[col].infer_objects(copy=False)
            df_ml[col] = df_ml[col].astype('boolean').fillna(False)  # Convertir a booleano y llenar NaN
            df_ml[col] = df_ml[col].astype(int)  # Convertir a entero
    
    # 11. Procesar precio
    # ----------------
    if 'precio_valor' in df_ml.columns:
        df_ml['precio_valor'] = pd.to_numeric(df_ml['precio_valor'], errors='coerce')
        df_ml['precio_gratuito'] = (df_ml['precio_valor'] == 0).astype(int)
        df_ml['precio_bajo'] = ((df_ml['precio_valor'] > 0) & (df_ml['precio_valor'] <= 10)).astype(int)
        df_ml['precio_medio'] = ((df_ml['precio_valor'] > 10) & (df_ml['precio_valor'] <= 30)).astype(int)
        df_ml['precio_alto'] = (df_ml['precio_valor'] > 30).astype(int)
    
    # 12. Agregar columna ciudad_madrid para uniformidad
    # ----------------------------------------------
    df_ml['ciudad_Madrid'] = 1
    
    # 13. Eliminar columnas originales que ya han sido procesadas
    # ------------------------------------------------------
    columnas_a_eliminar = [
        'título', 'descripción', 'categoría', 'subcategoría', 'ciudad',
        'día_días', 'distrito', 'fecha_inicio', 'fecha_fin', 'periodicidad',
    ]
    cols_to_drop = [col for col in columnas_a_eliminar if col in df_ml.columns]
    df_ml = df_ml.drop(columns=cols_to_drop)
    
    # 14. Opcional: Estandarización de variables numéricas
    # ------------------------------------------------
    # Identificar columnas numéricas para estandarización
    numeric_cols = df_ml.select_dtypes(include=[np.number]).columns
    
    # No estandarizamos columnas binarias (0/1)
    binary_cols = []
    for col in numeric_cols:
        if set(df_ml[col].dropna().unique()).issubset({0, 1}):
            binary_cols.append(col)
    
    # Columnas a estandarizar (numéricas que no son binarias)
    scale_cols = [col for col in numeric_cols if col not in binary_cols]
    
    # Aplicar StandardScaler a estas columnas
    if scale_cols:
        scaler = StandardScaler()
        df_ml[scale_cols] = scaler.fit_transform(df_ml[scale_cols])
    
    # 15. Manejo de valores faltantes finales
    # ------------------------------------
    # Rellenar los valores NaN restantes con 0
    df_ml = df_ml.fillna(0)

    
    return df_ml

In [93]:
# Llamado a la función para df_depurado2
import pandas as pd
#from df_to_ml import df_to_ml

# Supongamos que 'df_depurado2' es tu DataFrame original
# Código para cargar dicho dataframe (que tengo guardado) si no lo tenemos almacenado en una variable habiendo corrido todo el notebook anterior
# df_depurado2 = pd.read_csv('../data/processed/actividades(261)_detalles_depurado2_02mayo2025.csv')

# Aplicar la función de procesamiento
df_procesado_ml = df_to_ml(df_depurado2)

df_procesado_ml.head()

,edad_min,edad_max,requiere_inscripcion,hora_inicio,hora_fin,precio_valor,recomendado,tiene_url_info,tiene_url_imagen,titulo_longitud,descripcion_longitud,titulo_tfidf_0,titulo_tfidf_1,titulo_tfidf_2,titulo_tfidf_3,titulo_tfidf_4,desc_tfidf_0,desc_tfidf_1,desc_tfidf_2,desc_tfidf_3,desc_tfidf_4,categoría_Cultura y Espectáculos,categoría_Deporte,categoría_La Ciudad,categoría_Educación,categoría_Aire Libre,subcategoría_Club de lectura,subcategoría_Títeres y marionetas,subcategoría_Idiomas,subcategoría_Parques y jardines,subcategoría_Carreras y eventos,subcategoría_Verano,subcategoría_Música,subcategoría_Conciertos,subcategoría_Actividades al Aire Libre,subcategoría_Bailes,subcategoría_Talleres,subcategoría_Actividades,subcategoría_Magia,subcategoría_Circo,subcategoría_Actividades con animales,subcategoría_Ludotecas,subcategoría_Exposiciones,subcategoría_Campamentos Urbanos,subcategoría_Proyecciones,subcategoría_Cuentacuentos,subcategoría_Semana Santa,subcategoría_Teatro Infantil,edad_0_3,edad_4_6,edad_7_9,edad_10_12,edad_13_15,edad_16_18,fecha_inicio_year,fecha_inicio_month,fecha_inicio_day,fecha_inicio_dayofweek,fecha_inicio_is_summer,fecha_inicio_is_winter,fecha_inicio_is_weekend,fecha_fin_year,fecha_fin_month,fecha_fin_day,fecha_fin_dayofweek,fecha_fin_is_summer,fecha_fin_is_winter,fecha_fin_is_weekend,duracion_dias,duracion_corta,duracion_media,duracion_larga,hora_inicio_manana,hora_inicio_mediodia,hora_inicio_tarde,hora_inicio_noche,hora_fin_manana,hora_fin_mediodia,hora_fin_tarde,hora_fin_noche,dia_lunes,dia_martes,dia_miércoles,dia_jueves,dia_viernes,dia_sábado,dia_domingo,dia_findesemana,dia_entresemana,periodicidad_diaria,periodicidad_semanal,periodicidad_quincenal,periodicidad_mensual,distrito_Centro,distrito_Puente de Vallecas,distrito_Salamanca,distrito_Usera,distrito_Vicálvaro,distrito_Latina,distrito_Hortaleza,distrito_No especificado,distrito_San Blas-Canillejas,distrito_Tetuán,distrito_Moratalaz,distrito_Villa de Vallecas,distrito_Chamartín,distrito_Arganzuela,distrito_Retiro,distrito_Chamberí,distrito_Ciudad Lineal,distrito_Carabanchel,distrito_Barajas,distrito_Moncloa-Aravaca,distrito_Fuencarral-El Pardo,distrito_Villaverde,precio_gratuito,precio_bajo,precio_medio,precio_alto,ciudad_Madrid
0,0.034213,0.061509,0,-1.291316,-1.271095,-0.063246,1,0,1,0.127032,0.368911,-0.264120,-0.327036,-0.219039,-0.291002,0.0,0.0,-0.373065,-0.238063,0.0,-0.281300,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,-3.345217,2.378662,0.612021,1.048900,1,0,1,0.0,1.185701,0.501470,1.017355,1,0,1,3.527884,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,1.687835,-0.911454,1,0.875340,0.654059,-0.063246,1,1,0,0.905329,-0.727263,3.296384,1.935970,4.856233,-0.291002,0.0,0.0,-0.373065,-0.238063,0.0,3.707030,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-3.345217,2.378662,0.960709,-1.291129,1,0,0,0.0,1.185701,-0.819522,-1.548585,1,0,0,3.327768,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,2.018559,-0.911454,1,0.875340,0.654059,-0.063246,1,1,1,0.905329,2.777404,3.296384,1.935970,4.856233,-0.291002,0.0,0.0,-0.373065,4.114471,0.0,0.594915,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-3.345217,2.378662,1.541856,-2.461144,1,0,0,0.0,1.185701,-1.059702,-2.831556,1,0,0,3.227710,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,1.687835,-1.154695,1,0.720579,0.505970,-0.063246,1,1,1,1.372307,0.361191,2.859792,1.658479,4.233900,1.976820,0.0,0.0,-0.373065,-0.238063,0.0,-0.281300,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,-3.345217,2.378662,1.541856,-2.461144,1,0,0,0.0,1.185701,-1.900333,-2.831556,1,0,0,3.127652,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,2.018559,-0.911454,1,0.875340,0.654059,-0.063246,1,1,1,0.801556,2.777404,3.296384,1.935970,4.856233,

In [83]:
pd.set_option('display.max_columns', None)

In [94]:
df_procesado_ml.columns.tolist()

['edad_min',
 'edad_max',
 'requiere_inscripcion',
 'hora_inicio',
 'hora_fin',
 'precio_valor',
 'recomendado',
 'tiene_url_info',
 'tiene_url_imagen',
 'titulo_longitud',
 'descripcion_longitud',
 'titulo_tfidf_0',
 'titulo_tfidf_1',
 'titulo_tfidf_2',
 'titulo_tfidf_3',
 'titulo_tfidf_4',
 'desc_tfidf_0',
 'desc_tfidf_1',
 'desc_tfidf_2',
 'desc_tfidf_3',
 'desc_tfidf_4',
 'categoría_Cultura y Espectáculos',
 'categoría_Deporte',
 'categoría_La Ciudad',
 'categoría_Educación',
 'categoría_Aire Libre',
 'subcategoría_Club de lectura',
 'subcategoría_Títeres y marionetas',
 'subcategoría_Idiomas',
 'subcategoría_Parques y jardines',
 'subcategoría_Carreras y eventos',
 'subcategoría_Verano',
 'subcategoría_Música',
 'subcategoría_Conciertos',
 'subcategoría_Actividades al Aire Libre',
 'subcategoría_Bailes',
 'subcategoría_Talleres',
 'subcategoría_Actividades',
 'subcategoría_Magia',
 'subcategoría_Circo',
 'subcategoría_Actividades con animales',
 'subcategoría_Ludotecas',
 'subcate

______________________________________

Al final, guardamos el dataframe obtenido en processed.

In [97]:
# Guardar el DataFrame modificado (sin columna fechas)

# Definir ruta del archivo a guardar:
directorio = '../data/processed/'
nombre_archivo_procesado = 'actividades(261)_detalles_depuradoML_02mayo2025.csv'
ruta_archivo_trabajo = os.path.join(directorio, nombre_archivo_procesado)

df_procesado_ml.to_csv(ruta_archivo_trabajo, index=False, encoding='utf-8-sig')
print(f"Datos procesados guardados en carpeta '{ruta_archivo_trabajo}'")

Datos procesados guardados en carpeta '../data/processed/actividades(261)_detalles_depuradoML_02mayo2025.csv'



[Ir al siguiente notebook: Modelos Machine Learning](06_ModeloML_RecomencionActividades_KNN_agv.ipynb)